## Conformational analysis of a small molecule
Conformational analysis node modeled based on a workflow from Evotec

In [1]:
import BioSimSpace as BSS

In [2]:
node = BSS.Gateway.Node("A node to parameterise a small molecule, run MD and do a conformational analysis.")

In [3]:
node.addAuthor(name="Antonia Mey", email="antonia.mey@ed.ac.uk", affiliation="University of Edinburgh")
node.setLicense("GPLv3")

In [4]:
node.addInput("file", BSS.Gateway.File(help="A molecular input file, e.g. a PDB file."))

node.addInput("forcefield", BSS.Gateway.String(help="The name of the force field to use for parameterisation.",
                                               allowed=BSS.Parameters.forceFields(), default="gaff"))

node.addInput("water", BSS.Gateway.String(help="The name of the water model to use for solvation.",
                                          allowed=BSS.Solvent.waterModels(), default="tip3p"))

node.addInput("box_size", BSS.Gateway.Length(help="The base length of the cubic simulation box.", unit="nanometer"))

node.addInput("ion_conc", BSS.Gateway.Float(help="The ionic concentration in mol/litre.",
                                            minimum=0, maximum=1, default=0))
node.addInput("minimize", BSS.Gateway.String(help="Config file for minimisation"))
node.addInput("step2", BSS.Gateway.String(help="Config file for step2 equilibration"))
node.addInput("step3", BSS.Gateway.String(help="Config file for step3 equilibration"))
node.addInput("step4", BSS.Gateway.String(help="Config file for step4 equilibration"))
node.addInput("step5", BSS.Gateway.String(help="Config file for step5 equilibration"))
node.addInput("production", BSS.Gateway.String(help="Config file for production run"))


## Add optional output

In [5]:
node.addOutput("system", BSS.Gateway.FileSet(help="The parameterised and solvated molecular system in AMBER format."))

In [149]:
node.addOutput("equ5_out", BSS.Gateway.File(help="The output for equilibration 5 output."))


In [6]:
node.showControls()

Box(children=(Box(children=(Box(children=(Label(value='file: A molecular input file, e.g. a PDB file.'), FileU…

Uploaded '2-cyclopentanylindole.mol2' (2.97 kB)


## Setting the system up
The following reads and parametrises a small molecule and will save it the specified output. 

In [7]:
system = BSS.IO.readMolecules(node.getInput("file"))
molecule = system.getMolecules()[0]
molecule = BSS.Parameters.parameterise(molecule, node.getInput("forcefield")).getMolecule()
system = BSS.Solvent.solvate(node.getInput("water"), molecule=molecule,
                                                     box=3 * [node.getInput("box_size")],
                                                     ion_conc=node.getInput("ion_conc"))

node.setOutput("system", BSS.IO.saveMolecules("system", system, ["prm7", "rst7", "PDB"]))

## Step 1-5 of the equilibrations

### Minimisation

In [51]:
protocol = BSS.Protocol.Minimisation()
process = BSS.Process.Amber(system, protocol, name="minimize")
process.setConfig(node.getInput("minimize"))
process.setArg('-ref', 'minimize.rst7')

In [58]:
process.start()

BioSimSpace.Process.Amber(<BioSimSpace.System: nMolecules=2156>, BioSimSpace.Protocol.Minimisation(steps=10000), exe='/home/ppxasjsm/Software/amber18//bin/sander', name='minimize', work_dir='/tmp/tmpnjs83n2d', seed=None)

In [62]:
minimised = process.getSystem()

In [63]:
filenames = BSS.IO.saveMolecules("minimised", minimised, system.fileFormat())
print("\nWritten minimised molecular structure to: %s" % filenames)


Written minimised molecular structure to: ['/home/ppxasjsm/Projects/git/BioSimSpace/minimised.grotop', '/home/ppxasjsm/Projects/git/BioSimSpace/minimised.gro87']


### Equilibration 2

In [65]:
protocol = BSS.Protocol.Equilibration()
process = BSS.Process.Amber(minimised, protocol, name="equ2")
process.setConfig(node.getInput("step2"))

In [68]:
process.start()

BioSimSpace.Process.Amber(<BioSimSpace.System: nMolecules=2156>, BioSimSpace.Protocol.Equilibration(timestep=2.0000 fs, runtime=0.2000 ns, temperature_start=300.0000 K, temperature_end=300.0000 K, frames=20, ensemble='NVT', restrain_backbone=False), exe='/home/ppxasjsm/Software/amber18//bin/sander', name='equ2', work_dir='/tmp/tmpmf7e9shk', seed=None)

In [77]:
process.isError()

False

In [76]:
equ2 = process.getSystem()

### Equilibration 3

In [83]:
protocol = BSS.Protocol.Equilibration()
process = BSS.Process.Amber(equ2, protocol, name="equ3")
process.setConfig(node.getInput("step3"))

In [84]:
process.setArg('-ref', 'equ3.rst7')

In [85]:
process.start()

BioSimSpace.Process.Amber(<BioSimSpace.System: nMolecules=2156>, BioSimSpace.Protocol.Equilibration(timestep=2.0000 fs, runtime=0.2000 ns, temperature_start=300.0000 K, temperature_end=300.0000 K, frames=20, ensemble='NVT', restrain_backbone=False), exe='/home/ppxasjsm/Software/amber18//bin/sander', name='equ3', work_dir='/tmp/tmp0z58iy9a', seed=None)

In [95]:
process.isRunning()

False

In [96]:
process.isError()

False

In [97]:
equ3 = process.getSystem()

## Equilibration 4

In [98]:
protocol = BSS.Protocol.Equilibration()
process = BSS.Process.Amber(equ3, protocol, name="equ4")
process.setConfig(node.getInput("step4"))

In [99]:
process.setArg('-ref', 'equ4.rst7')

In [100]:
process.start()

BioSimSpace.Process.Amber(<BioSimSpace.System: nMolecules=2156>, BioSimSpace.Protocol.Equilibration(timestep=2.0000 fs, runtime=0.2000 ns, temperature_start=300.0000 K, temperature_end=300.0000 K, frames=20, ensemble='NVT', restrain_backbone=False), exe='/home/ppxasjsm/Software/amber18//bin/sander', name='equ4', work_dir='/tmp/tmptjvmxlr9', seed=None)

In [108]:
equ4 = process.getSystem()

## Equilibration 5

In [114]:
protocol = BSS.Protocol.Equilibration()
process = BSS.Process.Amber(equ3, protocol, name="equ5")
process.setConfig(node.getInput("production"))

In [115]:
process.start()

BioSimSpace.Process.Amber(<BioSimSpace.System: nMolecules=2156>, BioSimSpace.Protocol.Equilibration(timestep=2.0000 fs, runtime=0.2000 ns, temperature_start=300.0000 K, temperature_end=300.0000 K, frames=20, ensemble='NVT', restrain_backbone=False), exe='/home/ppxasjsm/Software/amber18//bin/sander', name='equ5', work_dir='/tmp/tmpu0_o2iv6', seed=None)

In [151]:
process.getOutput()

/home/ppxasjsm/Projects/git/BioSimSpace/equ5.zip

In [146]:
equ5 = process.getSystem()

## Production
Running the production MD simulation. This may take a while. 

In [ ]:
protocol = BSS.Protocol.Production()
process = BSS.Process.Amber(equ5, protocol, name="prod")
process.setConfig(node.getInput("production"))

In [ ]:
process.start()

In [ ]:
traj = process.getTrajectory()

## Conformational analysis using pytraj

In [ ]:
import pytraj as pt